In [2]:
%%html

<style>    
    @import url("css/custom_styles.css")
</style>

In [3]:
%matplotlib inline

#
# import utilitaires
#
import os

import helpers as hlp
import helpers.bovw as bovw
import helpers.features as feat

from sklearn.cluster import MiniBatchKMeans, DBSCAN

In [5]:
mbk_clusterer = MiniBatchKMeans(n_clusters=1000, batch_size=256 * os.cpu_count())

with hlp.Profile() as orb_kmeans_profile:
    mbk_clusterer.fit(orb_features.features)

print("ORB BoVW process time (MiniBatchKMeans):", orb_kmeans_profile.round_duration(), "s")
print( mbk_clusterer.cluster_centers_.shape )

ORB BoVW process time: 238.96 s
(1000, 32)
